In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# Enumerate SRA metadata

In [ ]:
sra_run_table_filepaths = [
    'SRP160434_SraRunTable.txt',
    'SRP160435_SraRunTable.txt'
]

geo_samples_filepaths = [
    'GSE119693_Samples.txt'
]

In [ ]:
organism_to_genome = {
    'Homo sapiens': 'hg38',
    'Mus musculus': 'mm10'
}

In [ ]:
sra_runs_df = (
    pd.concat([
        pd.read_csv(fp)
        for fp
        in sra_run_table_filepaths
    ]).merge(
        pd.concat([
            pd.read_csv(fp, sep = '\t')
            for fp
            in geo_samples_filepaths
        ])
    )
)


In [ ]:
sra_runs_df['Genome'] = sra_runs_df['Organism'].map(organism_to_genome)

In [ ]:
sra_runs_df[['GEO_Accession (exp)', 'Sample Name']]

In [ ]:
print('\n'.join(list(sra_runs_df.columns)))

# Download SRA runs as fastqs

In [ ]:
sra_runs_df['fastq_dir'] = 'fastqs/' + sra_runs_df['Genome']

In [ ]:
fasterq_dump_threads = 10
sra_runs_df['fasterq_dump_cmd'] = (
    f'mkdir -p ' + sra_runs_df['fastq_dir'] + ';' +
    f'fasterq-dump --outdir ' + sra_runs_df['fastq_dir'] + ' ' +
    f'--mem 4G --split-3 --threads {fasterq_dump_threads} ' + 
    f'--skip-technical  ' + 
    f'--print-read-nr ' + sra_runs_df['Run']
)

In [ ]:
%%time
for cmd in tqdm(list(sra_runs_df['fasterq_dump_cmd'])):
    print(cmd)
    # ! {cmd}

In [ ]:
parallel_jobs = 30
parallel_cmds_filepath = 'parallel_fasterq_dump_cmds.sh'
with open(parallel_cmds_filepath, 'w') as f:
    f.write('\n'.join(list(sra_runs_df['fasterq_dump_cmd']))+'\n')
parallel_cmd = f'cat {parallel_cmds_filepath}|parallel -j {parallel_jobs}'
print(parallel_cmd)
! {parallel_cmd}

# Cut adapters

In [ ]:
sra_runs_df['fastq_filepath'] = sra_runs_df['fastq_dir'] + '/' + sra_runs_df['Run'] + '.fastq'
sra_runs_df['trimmed_fastq_filepath'] = sra_runs_df['fastq_dir'] + '/' + sra_runs_df['Run'] + '_trimmed.fq.gz'

In [ ]:
trim_galore_threads = 8
sra_runs_df['trim_galore_cmd'] = (
    f'trim_galore -j {trim_galore_threads} ' +
    f'--fastqc --gzip ' +
    f'-o ' + sra_runs_df['fastq_dir'] + ' ' + 
    sra_runs_df['fastq_filepath'] + 
    f' &> ' + sra_runs_df['trimmed_fastq_filepath'] + '.log'
)

In [ ]:
for cmd in tqdm(list(sra_runs_df['trim_galore_cmd'])):
    print(cmd)
    ! {cmd}

# Generate genome index

In [ ]:
genomes = sorted(list(set(sra_runs_df['Genome'])))
genome_fa_filepaths = [
    f'genomes/{genome}/{genome}.fa'
    for genome
    in genomes
]


In [ ]:
genome_to_genome_fa_filepath = {
    k: v
    for k,v
    in zip(
        genomes,
        genome_fa_filepaths
    )
}

In [ ]:
bowtie2_threads = 30

In [ ]:
bowtie2_index_filepaths = [
    f'genomes/{genome}/{genome}.bowtie2/index'
    for genome
    in genomes
]

bowtie2_index_log_filepaths = [
    f'genomes/{genome}/{genome}.bowtie2.log'
    for genome
    in genomes
]


bowtie2_build_cmds = [
    (
        f'mkdir -p $(dirname {bowtie2_index_filepath});'
        f'bowtie2-build --threads {bowtie2_threads} {genome_fa_filepath} {bowtie2_index_filepath} '
        f'&> {bowtie2_index_log_filepath}'
    )
    for genome_fa_filepath, bowtie2_index_filepath, bowtie2_index_log_filepath
    in zip(
        genome_fa_filepaths,
        bowtie2_index_filepaths,
        bowtie2_index_log_filepaths
    )
]

In [ ]:
%%time
for cmd in tqdm(bowtie2_build_cmds):
    print(cmd)
    ! {cmd}

# Align reads

In [ ]:
genome_to_bowtie_index_filepath = {
    k: v
    for k,v
    in zip(
        genomes,
        bowtie2_index_filepaths
    )
}

In [ ]:
sra_runs_df['sam_filepath'] = (
    'alignments/' + sra_runs_df['Genome'] + '/' +
    sra_runs_df['Run'] + '.sam'
)
sra_runs_df['bowtie2_cmd'] = (
    f'mkdir -p alignments/' + sra_runs_df['Genome'] + ';'
    f'bowtie2 --threads {bowtie2_threads} -x ' +
    sra_runs_df['Genome'].map(genome_to_bowtie_index_filepath) + ' ' +
    f'-U ' + sra_runs_df['trimmed_fastq_filepath'] + ' ' +
    f'-S ' + sra_runs_df['sam_filepath'] +  ' ' +
    '&> ' + sra_runs_df['sam_filepath'] + '.log'
)

In [ ]:
for cmd in tqdm(list(sra_runs_df['bowtie2_cmd'])):
    print(cmd)
    ! {cmd}

# Make tag directories

In [ ]:
sra_runs_df['tagdir_basename'] = (
    sra_runs_df['Run'] + '_' + 
    sra_runs_df['Sample Title']
)

sra_runs_df['tagdir_filepath'] = (
    'tagdirs/' + sra_runs_df['Genome'] + '/' +
    sra_runs_df['tagdir_basename'] + '/'
)

In [ ]:
sra_runs_df['tagdir_cmd'] = (
    'mkdir -p ' + sra_runs_df['tagdir_filepath'] + ' ; '
    'makeTagDirectory ' +
    sra_runs_df['tagdir_filepath'] + ' ' +
     sra_runs_df['sam_filepath'] + ' ' +
    '-single -format sam '
    
)

In [ ]:
%%time
for cmd in tqdm(list(sra_runs_df['tagdir_cmd'])):
    print(cmd)
    ! {cmd}

# Simplify metadata

In [ ]:
samples_df = sra_runs_df[['Genome','tagdir_basename', 'tagdir_filepath']].copy()
samples_df['basename'] = samples_df['tagdir_basename']
samples_df['sample'] = True
samples_df['group'] = samples_df['basename'].map(lambda x: '_'.join(x.split('_')[1:-1]))
samples_df['replicate'] = samples_df['basename'].map(lambda x: int(x.split('_rep')[-1]))
samples_df

# Indicate comparison

In [ ]:
# group_1 = 'ATAC_UT'
# group_2 = 'ATAC_LPS-30min'

# group_1 = 'H3K27ac_UT'
# group_2 = 'H3K27ac_LPS-30min'

group_1 = 'ATAC_UT'
group_2 = 'ATAC_LPS-1h'

# group_1 = 'H3K27ac_UT'
# group_2 = 'H3K27ac_LPS-1h'

# group_1 = 'ATAC_UT'
# group_2 = 'ATAC_LPS-2h'

# group_1 = 'H3K27ac_UT'
# group_2 = 'H3K27ac_LPS-2h'

comparison_prefix = f'walkthrough.mouse_bmdm_lps_stim_atac.{group_1}.vs.{group_2}'

# comparison_prefix = f'walkthrough.mouse_bmdm_lps_stim_mnase.{group_1}.vs.{group_2}'

In [ ]:
tagdir_filepaths = list(samples_df[(samples_df['sample']) & (samples_df['group'].isin([group_1, group_2]))]['tagdir_filepath'])
input_tagdir_filepaths = list(samples_df[(samples_df['sample']==False) & (samples_df['group'].isin([group_1, group_2]))]['tagdir_filepath'])

genome = sorted(list(set(samples_df[samples_df['tagdir_filepath'].isin(tagdir_filepaths)]['Genome'])))[0]

tagdirs_str = ' '.join(tagdir_filepaths)
input_tagdirs_str = ' '.join(input_tagdir_filepaths)




# Call cleavage sites

In [ ]:
tss_min = 7

tagdirs_opt = f'-d {tagdirs_str}'
input_tagdirs_opt = ' ' if (len(input_tagdir_filepaths) == 0) else f'-dinput {input_tagdirs_str}'

tss_filepath = f'{comparison_prefix}.tss.min_raw_{tss_min}.txt'

get_tss_cmd = (
    f'perl getTSSfromReads.noLib.pl '
    f'{tagdirs_opt} '
    f'{input_tagdirs_opt} '
    f'-minRaw {tss_min} '
    f'> {tss_filepath}'
)


In [ ]:
%%time

for cmd in [get_tss_cmd]:
    print(cmd)
    ! {cmd}

# Quantify cleavage sites

In [ ]:
genome_filepath = f'genomes/{genome}/{genome}.fa'

In [ ]:
counts_filepath = tss_filepath[:-len('.txt')]+'.counts.txt'
counts_cmd = (
    f'annotatePeaks.pl {tss_filepath} '
    f'{genome_filepath} '
    f'-strand + -fragLength 1 -raw '
    f'-d {tagdirs_str} '
    f'> {counts_filepath}'
)

In [ ]:
rlogs_filepath = tss_filepath[:-len('.txt')]+'.rlog.txt'
rlogs_cmd = (
    f'annotatePeaks.pl {tss_filepath} '
    f'{genome_filepath} '
    f'-strand + -fragLength 1 -rlog '
    f'-d {tagdirs_str} '
    f'> {rlogs_filepath}'
)

In [ ]:
%%time

for cmd in [counts_cmd, rlogs_cmd]:
    print(cmd)
    ! {cmd}

In [ ]:
! head {counts_filepath}

In [ ]:
sample_basenames = list(samples_df[samples_df['sample']]['basename'])
group_1_basenames = [basename for basename in list(samples_df[samples_df['group'] == group_1]['basename']) if basename in sample_basenames]
group_2_basenames = [basename for basename in list(samples_df[samples_df['group'] == group_2]['basename']) if basename in sample_basenames]


In [ ]:
counts_df = pd.read_csv(counts_filepath, sep = '\t')

counts_df_col_renames = {
    col: col.split('/')[2]
    for col
    in (
        list(counts_df.columns)
        [
            -len(
                group_1_basenames + group_2_basenames
            ):
        ]
    )
}
# counts_df_col_renames[list(counts_df.columns)[0]] = 'PeakID'
counts_df = counts_df.rename(columns = counts_df_col_renames)
counts_df

# Select counts

In [ ]:
comparison_counts_filepath = counts_filepath[:-len('.txt')]+f'.comparison.txt'

In [ ]:
comparison_counts_df = counts_df.copy()
counts_df_col_renames_reverse = {v:k for k,v in counts_df_col_renames.items()}
comparison_counts_df_columns = (
    (
        list(comparison_counts_df.columns)
        [:-len(group_1_basenames + group_2_basenames)]
    ) + 
    group_1_basenames + 
    group_2_basenames
)
comparison_counts_df = comparison_counts_df[comparison_counts_df_columns].copy()
comparison_counts_df

In [ ]:
comparison_counts_df[group_1_basenames + group_2_basenames].sum()

In [ ]:
comparison_counts_df.fillna('NA').to_csv(comparison_counts_filepath, sep = '\t', index = False)

In [ ]:
print(comparison_counts_filepath)
! head {comparison_counts_filepath}

# Determine differential cleavage sites

In [ ]:
differential_filepath = comparison_counts_filepath[:-len('.txt')]+f'.differential.txt'

In [ ]:
basename_to_group_codes = samples_df[['basename', 'group']].copy().set_index('basename')['group'].to_dict()
basename_to_batch_codes = samples_df[['basename', 'replicate']].copy().set_index('basename')['replicate'].to_dict()

group_codes = [basename_to_group_codes[basename] for basename in (group_1_basenames + group_2_basenames)]
batch_codes = [basename_to_batch_codes[basename] for basename in (group_1_basenames + group_2_basenames)]

group_codes_str = ' '.join(map(str, group_codes))
batch_codes_str = ' '.join(map(str, batch_codes))

In [ ]:
differential_expression_cmd = (
    f'getDiffExpression.pl {comparison_counts_filepath} '
    f'{group_codes_str} '
    f'-batch {batch_codes_str} '
    f'> {differential_filepath}'
)


In [ ]:
%%time

print(differential_expression_cmd)
! {differential_expression_cmd}

In [ ]:
! head {differential_filepath}

In [ ]:
differential_df = pd.read_csv(differential_filepath, sep = '\t')
differential_df = differential_df.rename(columns = {list(differential_df.columns)[0]:'PeakID'})
differential_df

In [ ]:
differential_df['Name'] = differential_df['PeakID']
differential_df['log2fc'] = differential_df[list(differential_df.columns)[-4]]
differential_df['sum'] = differential_df['PeakID'].map(comparison_counts_df.copy().set_index(list(comparison_counts_df.columns)[0])[group_1_basenames + group_2_basenames].sum(axis = 1).to_dict())
differential_df = differential_df[differential_df['sum']>=tss_min].copy().reset_index(drop = True)

differential_df['Score'] = differential_df['log2fc']

differential_df.sort_values('log2fc', ascending = False)

# Write differential cleavage sites to BED file

In [ ]:
differential_bed_df = differential_df[['Chr', 'Start', 'End', 'Name', 'Score', 'Strand']].copy()
differential_bed_df[['Score']].hist(bins = 100)

In [ ]:
differential_bed_filepath = differential_filepath[:-len('.txt')]+'.log2fc.bed'
differential_bed_df.to_csv(differential_bed_filepath, sep = '\t', index = False, header = None)
! head {differential_bed_filepath}

# Cluster differential cleavage sites

In [ ]:
cluster_slop = 200
clustered_bed_filepath = differential_bed_filepath[:-len('.bed')]+f'.cluster_unstranded_slop_{cluster_slop}.bed'
cluster_cmd = f'bedtools sort -i {differential_bed_filepath} | bedtools cluster -d {cluster_slop} > {clustered_bed_filepath}'


In [ ]:
%%time

print(cluster_cmd)
! {cluster_cmd}

In [ ]:
clustered_differential_bed_df = pd.read_csv(clustered_bed_filepath, sep = '\t', header = None, names = ['Chr', 'Start', 'End', 'Name', 'Score', 'Strand', 'Cluster'])
clustered_differential_bed_df

# Deduplicate clusters, select site with highest sum
was "highest absolute score"

In [ ]:
cluster_deduplicated_differential_bed_df = clustered_differential_bed_df.copy()
cluster_deduplicated_differential_bed_df['Sum'] = cluster_deduplicated_differential_bed_df['Name'].map(differential_df[['Name', 'sum']].copy().set_index('Name')['sum'].to_dict())
cluster_deduplicated_differential_bed_df['Abs_Score'] = cluster_deduplicated_differential_bed_df['Score'].abs()
cluster_deduplicated_differential_bed_df['Rank_Score'] = cluster_deduplicated_differential_bed_df['Score'].rank(method = 'dense')

dedup_sort_col = 'Sum'
dedup_sort_ascending = False

cluster_deduplicated_differential_bed_df = (
    cluster_deduplicated_differential_bed_df
    .sort_values(by = ['Cluster', dedup_sort_col], ascending = [True, dedup_sort_ascending])
    .reset_index(drop = True)
    .copy()
    .drop_duplicates('Cluster')
    .reset_index(drop = True)
    .copy()
)
cluster_deduplicated_differential_bed_df[['Score']].hist(bins = 100)
plt.show()
cluster_deduplicated_differential_bed_df[['Rank_Score']].hist(bins = 100)
plt.show()
cluster_deduplicated_differential_bed_df

In [ ]:
score_type = 'asis'
score_col = 'Score'
# score_type = 'rank'

In [ ]:
if score_type == 'rank':
    score_col = 'Rank_Score'

In [ ]:
cluster_deduplicated_differential_bed_filepath = clustered_bed_filepath[:-len('.bed')]+f'.cluster_deduplicated.score_type_{score_type}.bed'
cluster_deduplicated_differential_bed_df[['Chr', 'Start', 'End', 'Name', score_col, 'Strand']].to_csv(cluster_deduplicated_differential_bed_filepath, sep = '\t', index = False, header = None)
! head {cluster_deduplicated_differential_bed_filepath}

# Run MEPP

In [ ]:
%%time
slop = 200
mepp_filepath = 'mepp_runs/'+cluster_deduplicated_differential_bed_filepath[:-len('.bed')]+f'.slop_{slop}.mepp'
motifs_filepath = 'homer.motifs.txt'
mepp_cmd = (
    f'bedtools slop -s -b {slop} -g {genome_filepath}.fai -i {cluster_deduplicated_differential_bed_filepath} '
    f'|python -m mepp.get_scored_fasta -fi {genome_filepath} '
    f'-bed - '
    f'|python -m mepp.cli '
    f'--fa - '
    f'--motifs {motifs_filepath} '
    f'--out {mepp_filepath} '
    f'--perms 200 '
    f'--batch 100 '
    f'--dgt 50 '
    f'--jobs 20 '
    f'--gjobs 10 '
    f'--nogpu '
    f'--dpi 100 '
    f'--orientations +,- '
    f'&> {mepp_filepath}.log'
)

In [ ]:
%%time
print(mepp_cmd)
! {mepp_cmd}

# Show MEPP links

In [ ]:
from IPython.display import display, Markdown


In [ ]:

mepp_results_table_md = f'[Results table]({mepp_filepath}/results_table_orientation_fwd.html)'
mepp_clustermap_md = f'[Clustermap]({mepp_filepath}/clustermap_orientation_fwd.html)'

In [ ]:
display(Markdown(mepp_results_table_md))
display(Markdown(mepp_clustermap_md))

# Run Centrimo

In [ ]:
peak_set_percent = 25
exclusion_percent = 100 - (peak_set_percent * 2)
top_percent = (100 - exclusion_percent//2)
bottom_percent = 100 - top_percent
print(top_percent)
print(bottom_percent)

In [ ]:
top_percentile_threshold = np.percentile(cluster_deduplicated_differential_bed_df['Score'], top_percent)
bottom_percentile_threshold = np.percentile(cluster_deduplicated_differential_bed_df['Score'], bottom_percent)
print(top_percentile_threshold)
print(bottom_percentile_threshold)

In [ ]:
top_cluster_deduplicated_differential_bed_df = cluster_deduplicated_differential_bed_df[cluster_deduplicated_differential_bed_df['Score']>=top_percentile_threshold]
bottom_cluster_deduplicated_differential_bed_df = cluster_deduplicated_differential_bed_df[cluster_deduplicated_differential_bed_df['Score']<=bottom_percentile_threshold]
print(top_cluster_deduplicated_differential_bed_df.shape)
print(bottom_cluster_deduplicated_differential_bed_df.shape)

In [ ]:
top_cluster_deduplicated_differential_bed_filepath = cluster_deduplicated_differential_bed_filepath[:-len('.bed')]+f'.top_{top_percent}_pct.bed'
bottom_cluster_deduplicated_differential_bed_filepath = cluster_deduplicated_differential_bed_filepath[:-len('.bed')]+f'.bottom_{bottom_percent}_pct.bed'

In [ ]:
top_cluster_deduplicated_differential_bed_df.to_csv(top_cluster_deduplicated_differential_bed_filepath, sep = '\t', index = False, header = None)
bottom_cluster_deduplicated_differential_bed_df.to_csv(bottom_cluster_deduplicated_differential_bed_filepath, sep = '\t', index = False, header = None)

In [ ]:
top_cluster_deduplicated_differential_fa_filepath = top_cluster_deduplicated_differential_bed_filepath[:-len('.bed')]+f'.slop_{slop}.fa'
bottom_cluster_deduplicated_differential_fa_filepath = bottom_cluster_deduplicated_differential_bed_filepath[:-len('.bed')]+f'.slop_{slop}.fa'

In [ ]:
top_fa_cmd = (
    f'bedtools slop -s -b {slop} -g {genome_filepath}.fai -i {top_cluster_deduplicated_differential_bed_filepath} '
    f'|python -m mepp.get_scored_fasta -fi {genome_filepath} '
    f'-bed - '
    f'> {top_cluster_deduplicated_differential_fa_filepath}'
)

In [ ]:
bottom_fa_cmd = (
    f'bedtools slop -s -b {slop} -g {genome_filepath}.fai -i {bottom_cluster_deduplicated_differential_bed_filepath} '
    f'|python -m mepp.get_scored_fasta -fi {genome_filepath} '
    f'-bed - '
    f'> {bottom_cluster_deduplicated_differential_fa_filepath}'
)

In [ ]:
for cmd in [top_fa_cmd, bottom_fa_cmd]:
    print(cmd)
    ! {cmd}

In [ ]:
meme_motifs_filepath = 'homer.motifs.id_fixed.meme'
centrimo_filepath = 'centrimo_runs/'+cluster_deduplicated_differential_bed_filepath[:-len('.bed')]+f'.top_vs_bottom_{peak_set_percent}_pct.slop_{slop}.centrimo'
centrimo_cmd = (
    f'mkdir -p {centrimo_filepath} ;'
    f'$(which time) --verbose '
    f'centrimo --oc {centrimo_filepath} '
    f'--neg {bottom_cluster_deduplicated_differential_fa_filepath} '
    f'--norc --sep --local --noseq '
    f'{top_cluster_deduplicated_differential_fa_filepath} '
    f'{meme_motifs_filepath}'
)

In [ ]:
print(centrimo_cmd)